In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
from pyspark.sql.types import StringType, ArrayType, LongType
from pyspark.sql.functions import col,udf,lower
import re
from pyspark.sql.functions import explode

#START start from here

In [103]:
df2 = spark.read.csv("gs://cluster-new/tmp_output/result.csv",header=True,mode="DROPMALFORMED")

In [105]:
df2.show()

+--------------------+--------------------+
|                From|                  To|
+--------------------+--------------------+
|https://www.nytim...|https://www.reute...|
|https://www.nytim...|https://www.reute...|
|https://www.nytim...|https://www.nytim...|
|https://www.nytim...|https://www.nytim...|
|https://www.nytim...|https://www.nytim...|
|https://www.nytim...|https://www.nytim...|
|https://www.nytim...|https://www.nytim...|
|https://www.nytim...|https://www.nytim...|
|https://www.activ...|http://www.cnn.co...|
|https://www.activ...|https://www.reute...|
|https://www.activ...|https://www.reute...|
|https://www.activ...|http://www.cnn.co...|
|https://www.activ...|http://www.cnn.co...|
|https://www.activ...|https://www.nytim...|
|https://www.activ...|https://www.reute...|
|https://www.nytim...|http://beforeitsn...|
|https://www.nytim...|http://beforeitsn...|
|https://www.nytim...|http://www.bbc.co...|
|http://dailybuzzl...|http://beforeitsn...|
|http://dailybuzzl...|http://bef

In [106]:
df = df2
# df.select('From').collect()

In [107]:
df_rdd = df.rdd.map(tuple)
# df_rdd.collect()

In [ ]:
df_rdd.count()

In [108]:
df_rdd = df_rdd.groupByKey()

In [ ]:
df_rdd.count()

In [109]:
# df_rdd.collect()

In [110]:
#this iterable dataframe might have 0 length list
df_rdd_filtered = df_rdd.filter(lambda x: len(x[1]) != 0)

# temp_df = sqlContext.createDataFrame(df_rdd.collect(), ['title', 'neighbors'])
# temp_df = temp_df.filter(len('neighbors') != 0)
# temp_df.show()

In [111]:
df_rdd_filtered.count()

528

In [112]:
#trying to get lenght of RHS column list
rdd_length = df_rdd.map(lambda x: len(x[1]))
# rdd_length.collect()

In [ ]:
# temp_df.count()

In [113]:
#test
# def test(df_rdd):
#     test_df = sqlContext.createDataFrame(df_rdd.collect(), ['title', 'rank'])
#     test_df.orderBy("rank").show()

# #passed
# test(df_rdd_filtered)

In [114]:
#initalize rank

ranks = df_rdd.map(lambda neighbors: (neighbors[0], 1.0))  
# ranks.collect()

In [ ]:
# ranks.count()

In [115]:
def calc_contrib(links, rank):
    num = len(links)
    for link in links:
        if num == 0:
            yield (link,0)
        else:
            yield (link, rank / num)

In [123]:
from operator import add
for i in range(1):
        # Calculates contributions
        contribs = df_rdd.join(ranks).flatMap(lambda (link, (links, rank)): calc_contrib(links, rank))
            # “x” means (key, (value list, 1.0)), therefore x[1][0] is the “value list” and x[1][1] is “1.0”. 
            #This function computes how much pagerank score a node receives from its neighbor.
             #reduceByKey(add) sums up the values for one key
        ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)  

In [124]:
# contribs.collect()  #contributions
contribs = df_rdd.join(ranks).flatMap(lambda x: calc_contrib(x[1][0], x[1][1]))
# test(contribs)

In [ ]:
# contribs.count()

In [125]:
#converting to df
l = ranks.collect()
df1 = sqlContext.createDataFrame(l, ['title', 'rank'])
df1.show()

+--------------------+-------------------+
|               title|               rank|
+--------------------+-------------------+
|http://www.bbc.co...|0.18207756003517162|
|https://www.reute...|0.21144579119615872|
|http://www.disclo...| 0.2111714958572234|
|http://www.cnn.co...| 0.1886914903975838|
|https://www.reute...|0.19019839904056873|
|http://beforeitsn...|0.21144578874816614|
|https://www.nytim...|0.23015827010706763|
|http://money.cnn....|0.18626627179734084|
|http://www.cnn.co...|0.17656535565436474|
|http://www.cnn.co...|0.22936666632909403|
|https://www.nytim...|0.17991601737281537|
|http://beforeitsn...| 0.2098319612890629|
|http://abcnews.go...|0.18626627179734084|
|https://www.reute...|0.17983093184786958|
|http://inhealth.c...|0.18207756003517162|
|https://www.nytim...|0.19874064773778782|
|https://www.reute...|0.18072289682207565|
|http://www.bbc.co...|0.18222145807184875|
|https://www.reute...|0.17983092932280006|
|http://www.bbc.co...|0.17991598064453146|
+----------

In [128]:
# save the entire csv file
df1.repartition(1).write.csv("gs://new_cluster/pagerank")